#Introducción

<!-- Aquí la idea es mostrarles como pueden usar BERT y quizás BETO en
español usando la librería Transformers. Me interesa que lo usen
de dos formas:

1) como extractor de vectores contextualizados

2) para hacer fine-tuning a otra task (e.g., Question Answering).
 -->


------------------------------------------------------
En esta auxiliar vamos a utilizar BERT, un modelo de lenguaje desarrollado por Google. Este modelo rompió varios récords en NLP y de hecho, cada vez que buscan en Google, BERT ayuda a refinar sus búsquedas.

## Pero, qué es BERT?

BERT es un personaje de plaza sésamo, al igual que ELMo, los cuales saben mucho de lenguaje y podemos ver con una mirada desafiante en la siguiente imagen:

![bert y elmo](https://i.imgur.com/1T4kyrq.png)

Los genios dándole nombres a los papers decidieron que era buena idea que los acrónimos se refirieran a los personajes de plaza sesamo, con los cuales algunos de nosotros (los más viejos) aprendimos a hablar y deletrear ~~hasta quizás mejor que en el jardín infantil~~. Al igual que Elmo, Embeddings from Language MOdels, BERT es el acrónimo de Bidirectional Encoder Representations from Transformers. \
Estos dos modelos producen "contextualized word embeddings". A diferencia de los modelos que producen static word embeddings como Word2Vec, la representación no depende solo de la palabra, sino que de la palabra y su contexto. Por lo tanto, cada palabra tiene infinitas representaciones, lo cual es mucho más flexible que tener solo un vector para cada palabra.




## Qué significa Bidirectional Encoder Representations from Transformers?
A diferencia de ELMo, el cual era una concatenación de información de izquierda-derecha y derecha-izquierda, BERT es bidireccional, es decir, toma en cuenta los contextos a la izquierda y derecha de la palabra simultáneamente.

BERT además utiliza Transformers, arquitecturas de deep learning altamente paralelizables que cuentan con un proceso de Encoder-Decoder. Dado que el objetivo de BERT es generar un modelo de lenguaje, solo es necesario el mecanismo de Encoding y le dejan el proceso de Decoding a las distintas tasks.


## Y como fue entrenado?

El primer objetivo de BERT es algo que se llama "masked language modeling". En este modelo, las palabras de una frase se borran al azar y se reemplazan por un token especial ([MASK]) con probabilidad 15%. Luego, se utiliza un Transformer para generar una predicción para la palabra remplazada por [MASK] basada en las palabras no enmascaradas que la rodean, tanto a la izquierda como a la derecha.

El segundo objetivo de BERT es resolver la tarea de Next Sentence Prediction. El modelo recibe dos oraciones como entrada y aprende a predecir si la segunda oración del par es la oración que siguiente del documento original. Durante el entrenamiento, el 50% de los inputs son un par en el que la segunda frase es la frase siguiente en el documento original, mientras que en el otro 50% se elige una frase aleatoria del corpus como segunda frase.


Pueden leer un poco más [acá](http://mlexplained.com/2019/01/07/paper-dissected-bert-pre-training-of-deep-bidirectional-transformers-for-language-understanding-explained/).



## Oye, pero esto suena un poco magico, tienes algunos ejemplos?

Hay bastantes librerías que tienen el modelo pre-entrenado a disposición, partiendo por el [GitHub de BERT](https://github.com/google-research/bert) implementado en TensorFlow. Como nosotros sabemos utilizar pytorch, utilizaremos la [version de HuggingFace](https://huggingface.co/transformers/) la cual es respaldada por el github de Google y la elogian: "which is compatible with our pre-trained checkpoints and is able to reproduce our results". Esta version se importa con la libreria transformers. Otras version disponibles son [sentence-bert](https://github.com/UKPLab/sentence-transformers) o [bert-as-service](https://github.com/hanxiao/bert-as-service).

In [1]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 34.1 MB/s 
     |████████████████████████████████| 596 kB 67.7 MB/s 
     |████████████████████████████████| 6.6 MB 59.6 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertTokenizer, BertModel
import torch

Veamos el primer ejemplo que entrega la documentación

In [3]:
# Si estamos utilizando google colab, no se preocupen por las descargas, ya que las hace el servidor de colab y no les gasta ancho de banda a uds
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Cargamos el tokenizador
model = BertModel.from_pretrained('bert-base-uncased') # Cargamos el modelo pre-entrenado

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 'pt' especifica que queremos vectores de pytorch, 'tf' seria en tensorflow
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt") 
# el doble asterico ayuda a evaluar los valores de un diccionario, por ejemplo:
# d = {'a': 1, 'b':2}, model(**d) sería equivalente a model(1,2)
outputs = model(**inputs)
# El ultimo hidden-state es el primer elemento del output del modelo.
last_hidden_states = outputs[0].squeeze(0) # squeeze en la primera dimension ya que es 1
print(inputs['input_ids']) # Tenemos 8 tokens, contando el CLS (101) y el SEP (102)
print(last_hidden_states.shape) # Tenemos 8 vectores de 768 dimensiones
print(last_hidden_states)

tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]])
torch.Size([8, 768])
tensor([[-0.1144,  0.1937,  0.1250,  ..., -0.3827,  0.2107,  0.5407],
        [ 0.5308,  0.3207,  0.3665,  ..., -0.0036,  0.7579,  0.0388],
        [-0.4877,  0.8849,  0.4256,  ..., -0.6976,  0.4458,  0.1231],
        ...,
        [-0.7003, -0.1815,  0.3297,  ..., -0.4838,  0.0680,  0.8901],
        [-1.0355, -0.2567, -0.0317,  ...,  0.3197,  0.3999,  0.1795],
        [ 0.6080,  0.2610, -0.3131,  ...,  0.0311, -0.6283, -0.1994]],
       grad_fn=<SqueezeBackward1>)


La primera pregunta es... cómo diantres obtengo una representación de mi oración desde el último hidden-state?

Las opciones más simples son tomar el token CLS, aunque no es muy recomendado, ya que depende del fine tunning (veremos esto más adelante) y la otra opción es tomar el promedio de todos mis tokens. Hay más formas de pooling (es decir como se mezclan los tokens), por ejemplo, podemos ver las de bert-as-service [acá](https://github.com/hanxiao/bert-as-service#q-what-are-the-available-pooling-strategies).

In [5]:
cantidad_tokens = inputs['input_ids'].shape[1]
# Representacion con token cls
cls_representation = last_hidden_states[0] # El primer token del ultimo hidden-state es el CLS
print(cls_representation.shape) # Representacion de 768 dimensiones

# Representacion con average 1 (más verbosa)
average = torch.zeros(768)
for i in range(1, cantidad_tokens-1): # Partimos en 1 y terminamos en largo-2 para ignorar CLS y SEP
  average += last_hidden_states[i] 
average = average/(cantidad_tokens-2) # Obtenemos nuestra representacion de 768 dimensiones
print(average.shape)

# Representacion con average 2 (más corta)
average2 = torch.mean(last_hidden_states[1:-1], 0)
print(average2.shape)
print(torch.equal(average, average2))

torch.Size([768])
torch.Size([768])
torch.Size([768])
True


## Ok, pero puedo representar oraciónes con BERT, puedo hacer algo más a parte de eso o se acabó la diversión?

¡Esto es solo el principio! ¡La librería transformers a parte nos presta modelos con decoders fine-tuneados en ciertas tareas y hasta podemos fine-tunearlos nosotros!

Fine-tunear ayuda a que BERT entienda cual es la tarea que queremos resolver. El modelo de por sí ya viene pre-entrenado en las 2 tareas que mencione previamente, Masked Language Modeling y Next Sentence Prediction sobre corpus muy grandes. 

Hay 2 modelos pre-entrenados de BERT: bert-base y bert-large que difieren en el tamaño del modelo, pero fueron entrenados sobre el mismo corpus: Wikipedia en ingles, además de aproximadamente 11.000 libros en ingles (esto se llama BookCorpus).

bert-base tiene 12 layers (transformer blocks), 12 attention heads, y 110 millones de parametros

bert-large tiene 24 layers (transformer blocks), 16 attention heads, y 340 millones de parametros

Una vez hice el cálculo rápido de cuanto me saldría entrenar bert-base desde cero en las cloud TPU de Google y era una cifra cercana a los 2000 dólares. Por suerte aquí tenemos a alguien que lo entrenó en español y nos puede contar su experiencia


## Gabriel y BETO

Experiencia de Gabriel y BETO

Continuando con los ejemplos, veamos como sería utilizar BertForNextSentencePrediction

In [6]:
from transformers import BertForNextSentencePrediction

Utilizamos el tokenizador común de BERT, solo cambiamos el modelo.

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Creemos una función que nos dice si tiene sentido o no la oración que continua.

In [38]:
def evaluar_oraciones(primera,segunda):
  encoding = tokenizer(primera, segunda, return_tensors='pt')
  outputs = model(**encoding, labels=torch.LongTensor([1])) # El label representa cual es la oración
  #Nota logits[0,0] entrega el score que la oracion si sea la siguiente (que tan True)
  #logits[0,1] entrega el score de que la oracion no sea la siguiente (que tan False)
  # Se puede aplicar una SoftMax sobre estos resultados para que sean probabilidades
  # Pero no es necesario.
  #return outputs
  if outputs.logits[0,0] < outputs.logits[0,1]:
    print("La oración no tiene nada que ver")
  elif outputs.logits[0,0] > outputs.logits[0,1]:
    print("La oración es una continuación")
  else:
    print("No estoy seguro")

Probemos este código con algunos ejemplos

In [39]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
evaluar_oraciones(prompt,next_sentence)

La oración no tiene nada que ver


In [40]:
prompt = "I'm really hungry."
next_sentence = "I'm getting a BigMac."
evaluar_oraciones(prompt,next_sentence)

La oración es una continuación


## Esto funciona bastante bien, que pasa si quiero entrenarlo para una tarea en especifico?

Para esto debemos fine-tunear el modelo con nuestros datos. Tomé [este](https://medium.com/swlh/painless-fine-tuning-of-bert-in-pytorch-b91c14912caa) tutorial como referencia por si algún paso no queda lo suficientemente claro. Vamos a fine-tunear BERT para realizar sentiment classification.

Lo primero es inicializar un modelo de BERT sin fine-tunning

In [13]:
from transformers import BertModel

In [14]:
#Creamos un modelo de BERT limpio
bert_model = BertModel.from_pretrained('bert-base-uncased')
#El mismo tokenizador de antes
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Debemos entender como se le pasan los datos a BERT. Imaginemos que queremos agregar varias oraciones simultaneamente. Como lo hacemos con tensores si tienen largo distinto? La solucion a esto se llama padding, es decir agregar tokens para que todas las secuencias tengan el mismo largo. \
Pero esto podría traer problemas si es que BERT llegase a interpretar estos tokens como partes de la oración verdad? Para eso es que es necesario especificarle a BERT cuales son los tokens a los que les tiene que tomar atención.

Por ejemplo, si tuviesemos un largo máximo de 12 tokens, para padear la oración 'I really enjoyed this movie a lot.' hariamos lo siguiente:

In [43]:
#Largo maximo de los tokens
T = 12
sentence = 'I really enjoyed this movie a lot.'
#Step 1: Tokenizar
tokens = tokenizer.tokenize(sentence) # ['i', 'really', 'enjoyed', 'this', 'movie', 'a', 'lot', '.']
#Step 2: Agregar [CLS] y [SEP]
tokens = ['[CLS]'] + tokens + ['[SEP]'] # ['[CLS]','i', 'really', 'enjoyed', 'this', 'movie', 'a', 'lot', '.', '[SEP]']
#Step 3: Padear tokens
padded_tokens = tokens + ['[PAD]' for _ in range(T - len(tokens))] #    ['[CLS]','i', 'really', 'enjoyed', 'this', 'movie', 'a', 'lot', '.', '[SEP]', '[PAD]', ... , '[PAD]']
attn_mask = [1 if token != '[PAD]' else 0 for token in padded_tokens] # [    1  , 1 ,    1    ,    1     ,   1   ,    1   ,  1 ,   1  ,  1 ,   1    ,    0   , ... ,    0   ] 
#Step 4: Segment ids: Estos representan cuando tienes 2 oraciones, la primera se llena con 0's y la segunda con 1's
seg_ids = [0 for _ in range(len(padded_tokens))] # En este caso no la usaremos, ya que es solo 1 oración. Su representacion son solo 0's
#Step 5: Cambiamos los tokens por su respectivo numero, CLS = 101, SEP = 102, etc...
token_ids = tokenizer.convert_tokens_to_ids(padded_tokens)

#Los cambiamos a tensores de pytorch antes de que entren al modelo, y es necesario agregarles una dimension extra.
# Esta dimension representa cuantas oraciones estamos pasando
token_ids = torch.tensor(token_ids).unsqueeze(0) #Shape : [1, 12]
attn_mask = torch.tensor(attn_mask).unsqueeze(0) #Shape : [1, 12]
seg_ids   = torch.tensor(seg_ids).unsqueeze(0) #Shape : [1, 12]

#Y al igual que antes podemos pasarselos a BERT
outputs= bert_model(token_ids, attention_mask = attn_mask, token_type_ids = seg_ids)
#outputs
print(outputs.last_hidden_state.shape)
#Out: torch.Size([1, 12, 768])
print(outputs.pooler_output.shape)
#Out: torch.Size([1, 768])

torch.Size([1, 12, 768])
torch.Size([1, 768])


No está de más agregar que BERT tiene un maximo de 512 tokens por input, por lo que si queremos agregar un texto muy grande debemos o truncarlo o separarlo en 2.\
Ahora que aprendimos como paddear oraciones, utilizaremos el Stanford Sentiment Tree Bank dataset que contiene movie reviews con sentimiento positivo (1) y negativo (0).\
Primero crearemos una clase para cargar los datos, extendiendo la clase Dataset que viene con pytorch:


In [16]:
from torch.utils.data import Dataset
import pandas as pd

class SSTDataset(Dataset):
    # Inicializacion de la clase
    def __init__(self, filename, maxlen):
        #Guardar los contenidos del dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')
        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # Establecer el largo máximo
        self.maxlen = maxlen

    # Funcion auxiliar que retorna el largo del dataframe
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        #Seleccionamos la oracion y el label de este dataset en especifico.
        sentence = self.df.loc[index, 'sentence']
        label = self.df.loc[index, 'label']

        #Realizamos todo el pre-procesamiento que explicamos anteriormente
        tokens = self.tokenizer.tokenize(sentence)
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        if len(tokens) < self.maxlen: # Comparamos con la cantidad maxima de tokens que dimos
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] # Si es mas corta agregamos padding
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]']  # Si es mas larga la cortamos

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Utilizamos el tokenizador para pasarlos a id
        tokens_ids_tensor = torch.tensor(tokens_ids) #Pasamos a tensor de pytorch

        #1 para los tokens no padeados, 0 si es padding
        attn_mask = (tokens_ids_tensor != 0).long()
        return tokens_ids_tensor, attn_mask, label

Creamos los dataloaders

In [45]:
from torch.utils.data import DataLoader

#Creamos instancias del training y validation sets
train_set = SSTDataset(filename = '/content/train.tsv', maxlen = 30)
val_set = SSTDataset(filename = '/content/dev.tsv', maxlen = 30)

#Creamos los dataloaders
train_loader = DataLoader(train_set, batch_size = 64, num_workers = 5)
val_loader = DataLoader(val_set, batch_size = 64, num_workers = 5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Ahora la parte más importante, generar nuestro modelo:

In [46]:
import torch.nn as nn
class SentimentClassifier(nn.Module):
    def __init__(self, freeze_bert = True):
        super(SentimentClassifier, self).__init__()
        #Creamos una instancia de BERT sin entrenamiento previo
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased').cuda()
        
        #Con esto podemos bloquear el entrenamiento de BERT, para comparar incluyendo el entrenamiento de bert y sin
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
        
        #La capa para clasificar
        #La idea es transformar una representacion de BERT (768 dimensiones) en 1 o 0 que representa el sentimiento
        self.cls_layer = nn.Linear(768, 1).cuda()

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            seq : Tensor of shape [B, T] containing token ids of sequences
            attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''
        #Le pasamos el input al modelo BERT
        cont_reps, _ = self.bert_layer(seq, attention_mask = attn_masks)

        #Obtenemos la representacion del token CLS
        cls_rep = cont_reps[:, 0]

        #Pasamos el token CLS por la capa de clasificacion
        logits = self.cls_layer(cls_rep)

        return logits

Utilizaremos binary cross-entropy loss y un descenso de gradiente estocastico

In [47]:
import torch.optim as optim
#Creamos el classificador de sentimiento basado en BERT
net_freezed = SentimentClassifier(freeze_bert = True)
net_not_freezed = SentimentClassifier(freeze_bert = False)
criterion = nn.BCEWithLogitsLoss()
opti_freezed = optim.Adam(net_freezed.parameters(), lr = 2e-5)
opti_not_freezed = optim.Adam(net_not_freezed.parameters(), lr = 2e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

Agregamos funciones axuliares para medir el desempeño del entrenamiento:

In [48]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc
    
def evaluate(net, criterion, dataloader):
    net.eval() # Modo evaluacion del modelo, pesos no serán modificados
    mean_acc, mean_loss = 0, 0
    count = 0
    with torch.no_grad(): # Los gradientes no serán guardados tampoco
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

Y con la siguiente funcion entrenamos los parametros

In [49]:
def train(net, criterion, opti, train_loader, val_loader, epochs):
    for ep in range(epochs): # Iterador de las epocas
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Enviamos los tensores a la GPU
            seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()

            #Evaluamos nuestro modelo en la secuencia y la mask de atencion
            logits = net(seq, attn_masks)

            #Calculamos la loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagation
            loss.backward()

            #Optimization step
            # Ojo que si no tenemos freeze_bert en true, vamos a entrenar los parametros de bert tambien.
            opti.step()

            if (it + 1) % 100 == 0:
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss : {} Train Accuracy : {}".format(it+1, ep+1, loss.item(), acc))
        val_acc, val_loss = evaluate(net, criterion, val_loader)
        print("Epoch {} complete! Validation Accuracy : {}, Validation Loss : {}".format(ep+1, val_acc, val_loss))

Y finalmente entrenamos ambos modelos

In [50]:
epochs = 5
train(net_freezed, criterion, opti_freezed, train_loader, val_loader, epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored

In [ ]:
train(net_not_freezed, criterion, opti_not_freezed, train_loader, val_loader, epochs)

Iteration 100 of epoch 1 complete. Loss : 0.2919929623603821 Train Accuracy : 0.890625
Iteration 200 of epoch 1 complete. Loss : 0.4610022008419037 Train Accuracy : 0.796875
Iteration 300 of epoch 1 complete. Loss : 0.3382772207260132 Train Accuracy : 0.875
Iteration 400 of epoch 1 complete. Loss : 0.14136575162410736 Train Accuracy : 0.921875
Iteration 500 of epoch 1 complete. Loss : 0.0685427188873291 Train Accuracy : 0.984375
Iteration 600 of epoch 1 complete. Loss : 0.15615978837013245 Train Accuracy : 0.921875
Iteration 700 of epoch 1 complete. Loss : 0.2990710735321045 Train Accuracy : 0.875
Iteration 800 of epoch 1 complete. Loss : 0.06944824755191803 Train Accuracy : 0.984375
Iteration 900 of epoch 1 complete. Loss : 0.13161134719848633 Train Accuracy : 0.96875
Iteration 1000 of epoch 1 complete. Loss : 0.12836754322052002 Train Accuracy : 0.96875
Epoch 1 complete! Validation Accuracy : 0.9058035612106323, Validation Loss : 0.25508606327431543
Iteration 100 of epoch 2 complete.

Comparando ambos entrenamientos, pasamos de 82% a 88% en 5 epocas. El primer entrenamiento solo entrenamos la capa de clasificacion mientras que en el segundo tambien modificamos los parametros de BERT.

Este ultimo entrenamiento es lo que llamamos fine-tunning. Lo más importante de esto es que no necesitamos un super computador para poder mejorar las representaciones en una tarea especifica.